<a href="https://colab.research.google.com/github/JinHuiXu1991/Jin_DATA606/blob/main/ipynb/DATA606_Part2_CollaborativeRecommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Amazon Product Recommender Systems
## Author: Jin Hui Xu

##Collaborative Filtering 

In [48]:
!wget https://github.com/JinHuiXu1991/Jin_DATA606/blob/main/cleaned_data/cleaned_amazon_product.zip?raw=true

!wget https://github.com/JinHuiXu1991/Jin_DATA606/blob/main/cleaned_data/cleaned_amazon_review.zip?raw=true

--2022-04-07 00:58:03--  https://github.com/JinHuiXu1991/Jin_DATA606/blob/main/cleaned_data/cleaned_amazon_product.zip?raw=true
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/JinHuiXu1991/Jin_DATA606/raw/main/cleaned_data/cleaned_amazon_product.zip [following]
--2022-04-07 00:58:03--  https://github.com/JinHuiXu1991/Jin_DATA606/raw/main/cleaned_data/cleaned_amazon_product.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/JinHuiXu1991/Jin_DATA606/main/cleaned_data/cleaned_amazon_product.zip [following]
--2022-04-07 00:58:03--  https://raw.githubusercontent.com/JinHuiXu1991/Jin_DATA606/main/cleaned_data/cleaned_amazon_product.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133,

In [49]:
!pip install surprise

In [122]:
import pandas as pd
import numpy as np
from surprise import KNNBasic
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split as train_test_split2
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

In [51]:
review_df = pd.read_csv('/content/cleaned_amazon_review.zip?raw=true', compression='zip')

review_df.head()

,overall,reviewTime,reviewerID,asin,reviewText,summary,reviewTimeYear,reviewTimeMonth
0,5.0,2013-11-27,A3NHUQ33CFH3VM,1118461304,not one thing in this book seemed an obvious o...,clear on what leads to innovation,2013,11
1,5.0,2013-11-01,A3SK6VNBQDNBJE,1118461304,i have enjoyed dr alan gregerman s weekly blog...,becoming more innovative by opening yourself t...,2013,11
2,5.0,2013-10-10,A3SOFHUR27FO3K,1118461304,alan gregerman believes that innovation comes ...,the world from different perspectives,2013,10
3,5.0,2013-10-09,A1HOG1PYCAE157,1118461304,alan gregerman is a smart funny entertaining w...,strangers are your new best friends,2013,10
4,5.0,2013-09-07,A26JGAM6GZMM4V,1118461304,as i began to read this book i was again remin...,how and why it is imperative to engage learn f...,2013,9


In [52]:
#Remove all information except Reviewer ID, Product ID and rating
review_df = review_df[['reviewerID', 'asin', 'overall']]

In [53]:
product_df = pd.read_csv('/content/cleaned_amazon_product.zip?raw=true', compression='zip')

product_df.head()

,category,description,title,brand,feature,main_cat,date,price,asin,imageURLHighRes,dateYear,dateMonth,ori_title
0,"Appliances Refrigerators, Freezers & Ice Makers",NaN,tupperware freezer square round container set ...,tupperware,each 3 pc set includes two 7 8 cup 200 ml and ...,appliances,2008-11-19,NaN,7301113188,[],2008,11,Tupperware Freezer Square Round Container Set ...
1,"Appliances Refrigerators, Freezers & Ice Makers",2 x tupperware pure fresh unique covered cool ...,2 x tupperware pure amp fresh unique covered c...,tupperware,2 x tupperware pure fresh unique covered cool ...,appliances,2016-06-05,3.62,7861850250,['https://images-na.ssl-images-amazon.com/imag...,2016,6,2 X Tupperware Pure &amp; Fresh Unique Covered...
2,Appliances Parts & Accessories,NaN,the cigar moments of pleasure,the cigar book,NaN,amazon home,NaN,150.26,8792559360,['https://images-na.ssl-images-amazon.com/imag...,0,0,The Cigar - Moments of Pleasure
3,Appliances Parts & Accessories,multi purpost descaler especially suited to wa...,caraselle 2x 50g appliance descalene,caraselle,NaN,tools home improvement,2014-12-17,NaN,9792954481,['https://images-na.ssl-images-amazon.com/imag...,2014,12,Caraselle 2X 50G Appliance Descalene
4,Appliances Parts & Accessories Range Parts & A...,full gauge and size beveled edge furnished wit...,eaton wiring 39ch sp l arrow hart 1 gang chrom...,eaton wiring,returns will not be honored on this closeout i...,tools home improvement,2007-01-16,3.43,B00002N5EL,[],2007,1,EATON Wiring 39CH-SP-L Arrow Hart 1-Gang Chrom...


In [54]:
#Remove all information except Product ID and title
product_df = product_df[['asin']]

In [55]:
merged_df = review_df.merge(product_df, on='asin', how='inner')
merged_df.head()

,reviewerID,asin,overall
0,A3SHVDMM83IHJ4,B00002N7IL,5.0
1,A3TIWHNJXMSIU7,B00002N7IL,5.0
2,AVP16JFIT6LPL,B00002N7IL,5.0
3,A2J2T9LX2HC947,B00002N7IL,5.0
4,AJN26F4KMWD59,B00002N7IL,5.0


In [56]:
merged_df = merged_df.groupby("reviewerID").filter(lambda x: len(x) > 4)

In [57]:
merged_df.shape

(12196, 3)

In [58]:
#Assign X as the original ratings dataframe and y as the user_id column of ratings.
X = merged_df.copy()
y = merged_df['reviewerID']

#Split into training and test datasets, stratified along user_id
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify=y, random_state=42)

In [59]:
#Function that computes the RMSE
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def baseline(reviewer_id, product_id):
    return 3.0

In [60]:
def score(cf_model):
    
    #Construct a list of user-movie tuples from the testing dataset
    id_pairs = zip(X_test['reviewerID'], X_test['asin'])
    
    #Predict the rating for every user-movie tuple
    y_pred = np.array([cf_model(reviewer, product) for (reviewer, product) in id_pairs])
    
    #Extract the actual ratings given by the users in the test data
    y_true = np.array(X_test['overall'])
    
    #Return the final RMSE score
    return rmse(y_true, y_pred)

In [61]:
score(baseline)

1.8130899266079477

# Ratings Matrix

In [62]:
r_matrix = X_train.pivot_table(values='overall', index='reviewerID', columns='asin')

r_matrix

asin,9792954481,B00002N7HY,B00002N7IL,B00004SQHD,B00004YWK2,B00005OU6T,B00006IV17,B00007E7RZ,B00009W3GY,B00009W3HB,...,B01H9WQ9QA,B01HAA3L9Y,B01HBPGP28,B01HDXLRGC,B01HDXM0UE,B01HDZ4OOM,B01HHBKZTA,B01HHF1G5I,B01HHF1HQQ,B01HICGMWC
reviewerID,,,,,,,,,,,,,,,,,,,,,
A100WO06OQR8BQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A10CGYG65ZQDNE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A10GYWIFBQU7C6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A10I4NC8TY5NMM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A10IHLG5RZHRMV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZGJF9K92NXJS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AZJK0A7AZTSN8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AZJRLTZ0OVYG9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
#User Based Collaborative Filter using Mean Ratings
def cf_user_mean(reviewer_id, product_id):
    
    #Check if movie_id exists in r_matrix
    if product_id in r_matrix:
        #Compute the mean of all the ratings given to the movie
        mean_rating = r_matrix[product_id].mean()
    
    else:
        #Default to a rating of 3.0 in the absence of any information
        mean_rating = 3.0
    
    return mean_rating

In [64]:
#Compute RMSE for the Mean model
score(cf_user_mean)

1.2412637796442219

## Weighted Mean

In [65]:
r_matrix_dummy = r_matrix.copy().fillna(0)

In [66]:
r_matrix_dummy

asin,9792954481,B00002N7HY,B00002N7IL,B00004SQHD,B00004YWK2,B00005OU6T,B00006IV17,B00007E7RZ,B00009W3GY,B00009W3HB,...,B01H9WQ9QA,B01HAA3L9Y,B01HBPGP28,B01HDXLRGC,B01HDXM0UE,B01HDZ4OOM,B01HHBKZTA,B01HHF1G5I,B01HHF1HQQ,B01HICGMWC
reviewerID,,,,,,,,,,,,,,,,,,,,,
A100WO06OQR8BQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A10CGYG65ZQDNE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A10GYWIFBQU7C6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A10I4NC8TY5NMM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A10IHLG5RZHRMV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZGJF9K92NXJS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AZJK0A7AZTSN8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AZJRLTZ0OVYG9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [67]:
#Compute the cosine similarity matrix using the dummy ratings matrix
cosine_sim = cosine_similarity(r_matrix_dummy, r_matrix_dummy)

In [68]:
cosine_sim = pd.DataFrame(cosine_sim, index=r_matrix.index, columns=r_matrix.index)

cosine_sim

reviewerID,A100WO06OQR8BQ,A10CGYG65ZQDNE,A10GYWIFBQU7C6,A10I4NC8TY5NMM,A10IHLG5RZHRMV,A10PEXB6XAQ5XF,A10PJB4MA8ITZO,A10Y058K7B96C6,A10ZBR6O8S8OCY,A111W73TEATLBS,...,AZ3AI1RNUX3YB,AZ5WZ0KJKPFLC,AZ8F7N52QBHQ3,AZAFVSVXJETPO,AZCOWCNGA721W,AZGJF9K92NXJS,AZJK0A7AZTSN8,AZJRLTZ0OVYG9,AZKFIZ5IESLW3,AZKPBWYY14GK1
reviewerID,,,,,,,,,,,,,,,,,,,,,
A100WO06OQR8BQ,1.0,0.0,0.0,0.0,0.0,0.0,0.217597,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A10CGYG65ZQDNE,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A10GYWIFBQU7C6,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A10I4NC8TY5NMM,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A10IHLG5RZHRMV,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZGJF9K92NXJS,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AZJK0A7AZTSN8,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0
AZJRLTZ0OVYG9,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.153243,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [69]:
#User Based Collaborative Filter using Weighted Mean Ratings
def cf_user_wmean(reviewer_id, product_id):
    
    #Check if movie_id exists in r_matrix
    if product_id in r_matrix:
        
        #Get the similarity scores for the user in question with every other user
        sim_scores = cosine_sim[reviewer_id]
        
        #Get the user ratings for the product in question
        m_ratings = r_matrix[product_id]
        
        #Extract the indices containing NaN in the m_ratings series
        idx = m_ratings[m_ratings.isnull()].index
        
        #Drop the NaN values from the m_ratings Series
        m_ratings = m_ratings.dropna()
        
        #Drop the corresponding cosine scores from the sim_scores series
        sim_scores = sim_scores.drop(idx)
        
        #Compute the final weighted mean
        if sim_scores.sum() == 0.0:
          wmean_rating = 3.0
        else:
          wmean_rating = np.dot(sim_scores, m_ratings)/ sim_scores.sum()
    
    else:
        #Default to a rating of 3.0 in the absence of any information
        wmean_rating = 3.0
    
    return wmean_rating

In [70]:
score(cf_user_wmean)

1.5072479560642944

## KNN

In [79]:
merged_df

,reviewerID,asin,overall
2,AVP16JFIT6LPL,B00002N7IL,5.0
30,A11QC8XY4UEWI7,B00004SQHD,5.0
105,A21YS2PSI5APWP,B00004YWK2,5.0
159,A2MVWBVM0CIAQ4,B00004YWK2,5.0
201,ALXRKQ572JVN4,B00004YWK2,5.0
...,...,...,...
601450,A3BWAUC9HZ90HP,B01HAA3L9Y,4.0
601456,A187ZYXJND15XH,B01HAA3L9Y,5.0
601505,A2P9QU3Y6B765N,B01HDXM0UE,5.0
601514,A1YHDVQBFA2I9Y,B01HDXLRGC,5.0


In [72]:
#Define a Reader object
#The Reader object helps in parsing the file or dataframe containing ratings
reader = Reader()

#Create the dataset to be used for building the filter

data = Dataset.load_from_df(merged_df, reader)

#Define the algorithm object; in this case kNN
knn = KNNBasic()

#Evaluate the performance in terms of RMSE
cross_validate(knn, data, measures=['RMSE'], cv=10, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBasic on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    1.0185  1.0472  0.9794  0.9464  0.9278  0.9761  0.9732  0.9762  0.8707  0.9868  0.

{'fit_time': (0.15144681930541992,
  0.16699600219726562,
  0.16036534309387207,
  0.1436007022857666,
  0.1640455722808838,
  0.14369702339172363,
  0.15680432319641113,
  0.16283893585205078,
  0.14594793319702148,
  0.14416098594665527),
 'test_rmse': array([1.01853467, 1.04722615, 0.97940743, 0.94638565, 0.92777534,
        0.97610452, 0.97315209, 0.97616002, 0.87070999, 0.9867865 ]),
 'test_time': (0.25615692138671875,
  0.21416330337524414,
  0.22057223320007324,
  0.2577023506164551,
  0.23350977897644043,
  0.25205349922180176,
  0.22873902320861816,
  0.23498153686523438,
  0.22333812713623047,
  0.2448749542236328)}

In [73]:
#Define the SVD algorithm object
svd = SVD()

#Evaluate the performance in terms of RMSE
cross_validate(svd, data, measures=['RMSE'], cv=10, verbose=True)

Evaluating RMSE of algorithm SVD on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    0.8401  0.8181  0.8400  0.8315  0.8281  0.8822  0.9308  0.8045  0.7756  0.7986  0.8349  0.0419  
Fit time          0.64    0.63    0.63    0.64    0.63    0.63    0.65    0.84    0.90    0.73    0.69    0.09    
Test time         0.01    0.01    0.02    0.01    0.01    0.01    0.01    0.01    0.01    0.01    0.01    0.00    


{'fit_time': (0.6425125598907471,
  0.6277315616607666,
  0.6274149417877197,
  0.6379008293151855,
  0.6322252750396729,
  0.6270372867584229,
  0.6532201766967773,
  0.8385796546936035,
  0.8953721523284912,
  0.7275104522705078),
 'test_rmse': array([0.84006243, 0.81810746, 0.83999578, 0.83150617, 0.82805922,
        0.88216701, 0.93080264, 0.80453896, 0.77561375, 0.79864065]),
 'test_time': (0.011589288711547852,
  0.010516166687011719,
  0.016284465789794922,
  0.012255668640136719,
  0.010828733444213867,
  0.011785030364990234,
  0.009815454483032227,
  0.014803886413574219,
  0.011013269424438477,
  0.011485576629638672)}

In [133]:
data = Dataset.load_from_df(merged_df, reader)
trainset, testset = train_test_split2(data, test_size=.2, random_state=42)

# We'll use the famous SVD algorithm.
algo = SVD()

cross_validate(algo, data, measures=['RMSE'], cv=10, verbose=True)

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

Evaluating RMSE of algorithm SVD on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    0.7534  0.8973  0.8280  0.8625  0.8423  0.8001  0.9019  0.7934  0.8670  0.8095  0.8355  0.0453  
Fit time          0.72    0.68    0.65    0.63    0.63    0.63    0.64    0.64    0.64    0.65    0.65    0.03    
Test time         0.01    0.01    0.01    0.01    0.01    0.01    0.02    0.01    0.01    0.01    0.01    0.00    
RMSE: 0.8241


0.8241238681606824